In [1]:
# import glassdoor_scraping as gs
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import datetime

In [3]:
#jobs website

def get_jobs(job_title, location, num_jobs):
    path = "/Users/cristianying/Desktop/glassdoor_salary_proj/chromedriver"
    driver = webdriver.Chrome(path)
    url = 'https://www.glassdoor.com.hk/Job/index.htm'
    driver.get(url)
    # driver.close() = close the tab .quit close window
    #when searching try first: ID, name, class

    search_job = driver.find_element_by_id("KeywordSearch")
    search_location = driver.find_element_by_id("LocationSearch")
    search_job.send_keys(job_title)
    search_location.clear()
    search_location.send_keys(location)
    search_location.send_keys(Keys.RETURN)
    jobs = []
    now = datetime.datetime.now()
    #wait for page to load first before finding the element

    while len(jobs) < num_jobs:
        #wait for page to load
        time.sleep(3)
        try:
            driver.find_element_by_xpath(".//li[contains(@class, 'react-job-listing')]").click()
        except ElementClickInterceptedException:
            pass
        #wait to check if sign in pops up
        time.sleep(5) 

        try:
            driver.find_element_by_xpath(".//span[@alt='Close']").click()  #clicking to the X.
        except NoSuchElementException:
            pass
        try:
            job_list = WebDriverWait(driver, 30).until(
                EC.presence_of_all_elements_located((By.XPATH, ".//li[contains(@class, 'react-job-listing')]"))
            )
            for company in job_list:
                print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
                if len(jobs) >= num_jobs:
                    break
                company.click()  #You might 
                collected_successfully = False

                #these elements exist for all companies, will wait until page loaded and data is retrieved
                while not collected_successfully:
                    try:
                        job_insights = driver.find_element_by_xpath("/html/body/div[3]/div/div/div[1]/div/div[2]/section/div/div/article/div/div[1]/div/div/div[3]/div[2]")

                        company_name = driver.find_element_by_xpath(".//div[@class = 'css-xuk5ye e1tk4kwz5']").text
                        job_type = job_insights.find_element_by_xpath(".//div[1]").text
                        industry = job_insights.find_element_by_xpath(".//div[3]").text
                        company_size = job_insights.find_element_by_xpath(".//div[4]").text
                        collected_successfully = True
                    except:
                        #wait until company div shows up
                        time.sleep(1)

                #below are separated from the while loop, in some cases they do not exist
                try:
                    founded = driver.find_element_by_xpath(".//span[contains(text(), 'Founded')]/following-sibling::span").get_attribute('innerHTML')
                except NoSuchElementException: 
                    founded = now.year
                try:
                    company_type = driver.find_element_by_xpath(".//span[contains(text(), 'Type') and @class = 'css-1pldt9b e1pvx6aw1']/following-sibling::span").get_attribute('innerHTML')
                except NoSuchElementException: 
                    company_type = 'NA'
                try:
                    sector = driver.find_element_by_xpath(".//span[contains(text(), 'Sector') and @class = 'css-1pldt9b e1pvx6aw1']/following-sibling::span").get_attribute('innerHTML')
                except NoSuchElementException: 
                    sector = 'NA'
                try:
                    rating = driver.find_element_by_xpath(".//div[@class = 'mr-sm css-ey2fjr e1pr2f4f3']").text
                except NoSuchElementException: 
                    rating = -1
                try:
                    base_salary = company.find_element_by_xpath(".//span[@data-test='detailSalary']").text                    
                except NoSuchElementException:
                    base_salary = -1 #You need to set a "not found value. It's important."

                #add job details to array
                jobs.append(
                    {
                        "company_name" : company_name,
                        "job_type" : job_type,
                        "base_salary" : base_salary,
                        "industry" : industry,
                        "company_size" : company_size,
                        "founded" : founded,
                        "rating" : rating,
                        "company_type" : company_type,
                        "sector" : sector
                    }
                )
            try:
                driver.find_element_by_xpath(".//a[@data-test='pagination-next']").click()
            except NoSuchElementException: 
                print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(30, len(jobs)))
                break
        
        except BaseException as err:
            print(f"Unexpected {err=}, {type(err)=}")
        

    #     time.sleep(5)

    #     try:
    #         driver.find_element_by_xpath(".//span[@alt='Close']").click()
    #     except NoSuchElementException: 
    #         print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(30, len(jobs)))
    #         break

    return pd.DataFrame(jobs)

In [8]:
df_jobs = get_jobs("data analyst", "Hong Kong", 100)
df_jobs

Progress: 0/100
Progress: 1/100
Progress: 2/100
Progress: 3/100
Progress: 4/100
Progress: 5/100
Progress: 6/100
Progress: 7/100
Progress: 8/100
Progress: 9/100
Progress: 10/100
Progress: 11/100
Progress: 12/100
Progress: 13/100
Progress: 14/100
Progress: 15/100
Progress: 16/100
Progress: 17/100
Progress: 18/100
Progress: 19/100
Progress: 20/100
Progress: 21/100
Progress: 22/100
Progress: 23/100
Progress: 24/100
Progress: 25/100
Progress: 26/100
Progress: 27/100
Progress: 28/100
Progress: 29/100
Progress: 30/100
Progress: 31/100
Progress: 32/100
Progress: 33/100
Progress: 34/100
Progress: 35/100
Progress: 36/100
Progress: 37/100
Progress: 38/100
Progress: 39/100
Progress: 40/100
Progress: 41/100
Progress: 42/100
Progress: 43/100
Progress: 44/100
Progress: 45/100
Progress: 46/100
Progress: 47/100
Progress: 48/100
Progress: 49/100
Progress: 50/100
Progress: 51/100
Progress: 52/100
Progress: 53/100
Progress: 54/100
Progress: 55/100
Progress: 56/100
Progress: 57/100
Progress: 58/100
Progres

,company_name,job_type,base_salary,industry,company_size,founded,rating,company_type,sector
0,"ExodusPoint Capital Management Hong Kong, Limi...",Job Type: Full-time,HK$210K - HK$368K (Glassdoor Est.),Industry: Finance,Size: 501 to 1000 Employees,2017,3.2,Company - Private,Finance
1,Achievers Recruitment Limited\n4.7,Job Type: Full-time,-1,Industry: Business Services,Size: 1 to 50 Employees,2016,4.7,Company - Private,Business Services
2,Excel Technology International (Hong Kong) Lim...,Job Type: Full-time,-1,Industry: Information Technology,Size: 201 to 500 Employees,1988,1.7,Company - Private,Information Technology
3,Sun Life Hong Kong Ltd\n4.5,Job Type: Full-time,-1,Industry: N/A,Size: Unknown,2021,4.5,Company - Private,NA
4,"Loscam (Asia Pacific) Co., Limited",Job Type: Full-time,-1,Industry: N/A,Size: N/A,2021,-1,NA,NA
...,...,...,...,...,...,...,...,...,...
95,Cignal Technology Company Limited,Job Type: Full-time,-1,Industry: Information Technology,Size: 1 to 50 Employees,2021,-1,Company - Private,Information Technology
96,Venturenix Limited\n2.0,Job Type: Full-time,-1,Industry: N/A,Size: 1 to 50 Employees,2021,2.0,Company - Private,NA
97,Taylor Coulter Limited\n5.0,Job Type: Full-time,-1,Industry: N/A,Size: Unknown,2021,5.0,Company - Private,NA
98,CEVA Logistics (Hong Kong) Limited\n3.6,Job Type: Full-time,-1,Industry: N/A,Size: 10000+ Employees,2006,3.6,Company - Private,NA


In [11]:
df_jobs.to_csv('glassdoor_jobs.csv', index = False)